In [1]:
import os
import numpy as np
import pandas as pd
from Bio import SeqIO

disprot_regions_file = "/data/franco/datasets/disprot/DisProt_release_2022_06_with_ambiguous_evidences.tsv"
df = pd.read_table(disprot_regions_file, header=0)

In [24]:
df

,acc,name,organism,ncbi_taxon_id,disprot_id,region_id,start,end,term_namespace,term,ec,reference,region_sequence,confidence,obsolete
0,P03265,DNA-binding protein,Human adenovirus C serotype 5,28285,DP00003,DP00003r002,294,334,Structural state,IDPO:00076,ECO:0006220,pmid:8632448,EHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNT,NaN,NaN
1,P03265,DNA-binding protein,Human adenovirus C serotype 5,28285,DP00003,DP00003r004,454,464,Structural state,IDPO:00076,ECO:0006220,pmid:8632448,VYRNSRAQGGG,NaN,NaN
2,P49913,Cathelicidin antimicrobial peptide,Homo sapiens,9606,DP00004,DP00004r001,134,170,Structural state,IDPO:00076,ECO:0006206,pmid:9452503,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,NaN,NaN
3,P49913,Cathelicidin antimicrobial peptide,Homo sapiens,9606,DP00004,DP00004r002,134,170,Structural transition,IDPO:00050,ECO:0006206,pmid:9452503,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,NaN,NaN
4,P49913,Cathelicidin antimicrobial peptide,Homo sapiens,9606,DP00004,DP00004r004,134,170,Biological process,GO:0019835,ECO:0007634,pmid:9452503,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10332,P40939,"Trifunctional enzyme subunit alpha, mitochondrial",Homo sapiens,9606,DP03726,DP03726r001,637,647,Structural state,IDPO:00076,ECO:0006003,pmid:29915090,YIYQEGVKRKD,NaN,NaN
10333,Q6CSX2,Serine/threonine-protein kinase ATG1,Kluyveromyces lactis (strain ATCC 8585 / CBS 2...,284590,DP03728,DP03728r001,562,831,Structural state,IDPO:00076,ECO:0006236,pmid:25139988,DSNITPAVESLAAKAFVMYSFAEMKFSQILPTPPSSTDYDPLSDKR...,NaN,NaN
10334,Q6CSX2,Serine/threonine-protein kinase ATG1,Kluyveromyces lactis (strain ATCC 8585 / CBS 2...,284590,DP03728,DP03728r002,562,831,Cellular component,GO:1990316,ECO:0006236,pmid:25139988,DSNITPAVESLAAKAFVMYSFAEMKFSQILPTPPSSTDYDPLSDKR...,NaN,NaN
10335,Q8IYT8,Serine/threonine-protein kinase ULK2,Homo sapiens,9606,DP03729,DP03729r001,168,177,Structural state,IDPO:00076,ECO:0006220,pmid:30782972,NMMAATLCGS,NaN,NaN


In [25]:
disprot_annotations_file = "/data/franco/datasets/disprot/DisProt_release_2022_06_with_ambiguous_evidences_consensus_regions.tsv"

df_annot = pd.read_table(disprot_annotations_file, header=0)
annot_dict = dict(zip(df_annot['acc'], df_annot['consensus']))

In [34]:
dataset_dict = dict()

for i in range(df.shape[0]):
#     print(df.iloc[i])
    uniprot = df.iloc[i]["acc"]
    seq     = df.iloc[i]["region_sequence"]
    regionid= df.iloc[i]["region_id"]
    start   = int(df.iloc[i]['start']) - 1
    end     = int(df.iloc[i]['end'])
    
    if uniprot in dataset_dict:
        if regionid in dataset_dict[uniprot]:
            print(f"Region {regionid} exists?")
            raise
        else:
            found = False
            for region in dataset_dict[uniprot]:
                if seq == dataset_dict[uniprot][region]['seq']:
                    found = True
                    dataset_dict[uniprot][region]['ids'].append(regionid)
            if not found:
                dataset_dict[uniprot][regionid] = dict()
                dataset_dict[uniprot][regionid]["seq"] = seq
                dataset_dict[uniprot][regionid]["ids"] = [regionid]
                dataset_dict[uniprot][regionid]["consensus"] = annot_dict[uniprot][start:end]
    else:
        dataset_dict[uniprot] = dict()
        dataset_dict[uniprot][regionid] = dict()
        dataset_dict[uniprot][regionid]["seq"] = seq
        dataset_dict[uniprot][regionid]["ids"] = [regionid]
        dataset_dict[uniprot][regionid]["consensus"] = annot_dict[uniprot][start:end]

In [36]:
with open("/data/franco/datasets/disprot/disprot_regions_seq.fasta", 'w') as outf:
    with open("/data/franco/datasets/disprot/disprot_regions_annot.fasta", 'w') as outg:
        for unip in dataset_dict:
            for region in dataset_dict[unip]:
                outf.write(f">{unip}_{region}|{','.join(dataset_dict[unip][region]['ids'])}\n")
                outg.write(f">{unip}_{region}|{','.join(dataset_dict[unip][region]['ids'])}\n")
                outf.write(f"{dataset_dict[unip][region]['seq']}\n")
                outg.write(f"{dataset_dict[unip][region]['consensus']}\n")